<a href="https://colab.research.google.com/github/barrosm/BIMASTER_CODE_PUBLIC/blob/main/005_leitura_dados_IPEADATA_BCB_rev_20210515.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leitura dados IPEADATA e Banco Central

## Monica Barros - 25/02/2021 - atualização em 15/05/2021

In [2]:
# para baixar os dados do ipeadata
#====================================================
!pip install ipeadatapy
import ipeadatapy as idpy

* ### Tive que usar o Quandl para baixar os dados - o módulo sgs não funcionava por causa do locale
* ### Também não funcionou ao tentar fazer a request direto e ler o json (às vezes ia, às vezes não ia)

In [3]:
# Alternativa para baixar os dados do Banco Central
#====================================================
!pip install quandl
import quandl
quandl.ApiConfig.api_key = 'Bs-h_k6jgX2cHYaYh2Cc'   # Obtenha sua própria key

In [4]:
# para baixar os dados diretamente do Banco Central
#===================================================
!pip install sgs
import sgs

     |████████████████████████████████| 122kB 3.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('fivethirtyeight')
pd.set_option('max_colwidth', 200)  # Altera largura máxima das colunas
pd.set_option('display.max_colwidth', None)

import seaborn as sns; 
sns.set(rc={'figure.figsize':(17,6)})

plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

import os
import datetime
from dateutil.relativedelta import relativedelta
import time
from time import perf_counter

import glob
import pickle

#!pip install openpyxl
from openpyxl import Workbook

#!pip install pytz
import pytz
local_tz = pytz.timezone('America/Sao_Paulo')

In [6]:
import urllib3
import json
import urllib.request

from pandas.io.json import json_normalize

In [7]:
path_result = '/home/jovyan/work/@Fund_Eval/@fatores_risco/'
os.makedirs(path_result, exist_ok = 'TRUE')

path_data = '/home/jovyan/work/@Fund_Eval/@dados_acoes/'
os.makedirs(path_data, exist_ok = 'TRUE')

path_code = '/home/jovyan/work/@Fund_Eval/PROGRAMAS/'
os.makedirs(path_code, exist_ok = 'TRUE')

In [8]:
def restringe_periodo(serie, ano_inicio):
    result = (idpy.timeseries(serie))
    if 'YEAR' in result:
        result = result.loc[result['YEAR'] >= ano_inicio]  # em outras séries a coluna é 'ANO'
    if 'ANO'  in result:
        result = result.loc[result['ANO'] >= ano_inicio] 
    return result

## Selecionando séries diárias no IPEADATA

- ANBIMA366_TJTLN são as taxas de juros pré de 1, 3, 6 e 12 meses
- BMF_366FUTXDOL = dolar futuro 1 e 3 meses
- BMF366_FUT3DOLV366 = volatilidade do dólar futuro, Calculada pelo Ipeadata com base nas taxas diárias de retorno entre as cotações de fechamento observadas nos 21 últimos dias de negociação
- GM366_ERC366 = Taxa de câmbio R /𝑈𝑆  comercial (valor de compra)
- GM366_ERTUR366 = Taxa de câmbio - R /𝑈𝑆  - turismo - venda - média
- GM366_ERV366 = Taxa de câmbio - R /𝑈𝑆  - comercial - venda - média
- GM366_ERVV366	= Taxa de câmbio - R /𝑈𝑆  - comercial - venda - média - volatilidade
- GM366_EUROV366 = Zona do Euro - taxa de câmbio - R$ / euro - venda - média
- EIA366_PBRENT366	www.eia.doe.gov	Preço - pétroleo bruto - Brent (FOB)
- EIA366_PWTI366	www.eia.doe.gov	Preço - pétroleo bruto - WTI (FOB)
- GM366_DOW366	www.valor.com.br	Índice de ações - Dow Jones - fechamento
- SGS366_NASDAQ366	www.nasdaq.com	Índice de ações - NASDAQ - fechamento
- VALOR366_FEDFUND366	www.valor.com.br	Taxa de juros básica - Federal Funds - fixada pelo FOMC
- GM366_IBVSP366	www.bmfbovespa.com.br	Índice de ações - Ibovespa - fechamento
- GM366_IBVSPV366	www.ipea.gov.br	Índice de ações - Ibovespa - fechamento - volatilidade


In [9]:
series_eco = ['ANBIMA366_TJTLN1366', 'ANBIMA366_TJTLN3366', 'ANBIMA366_TJTLN6366', 'ANBIMA366_TJTLN12366', 
             'BMF366_FUT1DOL366', 'BMF366_FUT3DOL366', 'BMF366_FUT3DOLV366', 
              'GM366_ERTUR366', 'GM366_ERV366', 'GM366_ERVV366', 'GM366_EUROV366', 
              'EIA366_PBRENT366', 'EIA366_PWTI366', 'GM366_DOW366', 'SGS366_NASDAQ366', 'VALOR366_FEDFUND366',
              'GM366_IBVSP366', 'GM366_IBVSPV366']

In [10]:
len(series_eco)

18

### Capturando os dados através do Ipeadata

In [ ]:
time_start = time.time()
time_start;

df1 = [restringe_periodo(serie, 2017) for serie in series_eco] 
df1

for i in range(len(series_eco)): 
    # print(i)
    # print(df1[i].columns)
    # Renomeia as colunas
    df1[i].columns = ['YEAR', 'DAY', 'MONTH', 'CODE', 'RAW DATE', 'VALUE ((% a.a.))']
    df1[i].rename(columns = {'VALUE ((% a.a.))':'VALUE'}, inplace = True)
    # Seleciona apenas as colunas que interessam
    df1[i] = df1[i][['CODE','YEAR', 'MONTH', 'VALUE']]

time_end = time.time();
elapsed_time = time_end - time_start;
print('')
print('*********************************************************************')
print('Demorou:',round(elapsed_time,2),' segundos')
print('*********************************************************************')

In [ ]:
# Cria Dataframe com as colunas concatenadas
df_eco = pd.concat([df1[i] for i in range(len(series_eco))], axis =1)
df_eco.head(3)

In [ ]:
df_eco.shape

In [ ]:
#df_eco.columns.to_list()

In [ ]:
df_eco = df_eco.iloc[:,np.r_[1:4, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63, 67, 71]]

In [ ]:
cols_dataframe = [str(i) for i in series_eco]
df_eco.columns = ['YEAR', 'MONTH'] + cols_dataframe

In [ ]:
## Renomeando o índice de df_eco (dados do Ipeadata) - CUIDADO - substitui o Dataframe original!
df_eco_back = df_eco.copy()  # por via das dúvidas
df_eco.rename_axis(index={'DATE': 'Date'}, inplace = True)
df_eco.head(3)

In [ ]:
df_eco.tail(3)

In [ ]:
# Volatilidades do Futuro 3 meses Dólar e Dólar Spot
df_eco[['BMF366_FUT3DOLV366', 'GM366_ERVV366']].plot();

In [ ]:
#df_eco[['BMF366_FUT1DOL366', 'BMF366_FUT3DOL366', 'GM366_ERV366']].plot();

In [ ]:
#df_eco[['ANBIMA366_TJTLN1366', 'ANBIMA366_TJTLN12366']].plot();

## Selecionando séries diárias no Banco Central

- 12461 = IRF-M (ANBIMA Indice Renda Fixa (pré))
- 12462 = IMA-S (Selic)
- 12463 = IMA-C (IGP-M - será descontinuado em 04/2021)
- 12466 = IMA-B (NTN-B, IPCA)
- 12467 = IMA-B 5 (NTN-B < 5 anos)
- 12468 = IMA-B 5+ 
- 12469 = IMA-Geral
- 17626 = IRF - M1 (pré < 1 ano)
- 17627 = IRF - M1+ (pré > 1 ano)
- 17628 = IMA Geral ex C
Fontes: Abecip e BCB-Depec

for i in lista
df_bacen = sgs.time_serie(CDI_CODE, start='02/01/2018', end='31/12/2018')

from datetime import datetime
agora = datetime.now(pytz.timezone('America/Sao_Paulo'))
agora = agora.date()
agora = str(agora)
agora

In [ ]:
lista_codigos = [12461, 12462, 12463, 12466, 12467, 12468, 12469, 17626, 17627, 17628]

In [ ]:
def importa_bacen(codigo, start_date = '01/01/2017', end_date = '31/12/2020'):
    '''usa o módulo sgs para importar dados do Bacen
    input é o código da série no Bacen''' 
    tt1 = datetime.datetime.strptime(start_date, '%d/%m/%Y')#'%Y-%m-%d')
    tt2 = datetime.datetime.strptime(end_date, '%d/%m/%Y')
    result = sgs.time_serie(codigo, start= tt1, end= tt2)
    return result

importa_bacen(12462)   ---- Não funciona

In [ ]:
start_date = '2017-01-01'
end_date = '2021-12-31'

In [ ]:
codigo_bc = 17628

In [ ]:
def consulta_bc2(codigo_bcb):
    #url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
    url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.'+str(codigo_bc)+'/dados?formato=json&dataInicial='+\
    str(start_date)+'&dataFinal='+str(end_date)
    
    df = pd.read_json(url)
    df['data'] = pd.to_datetime(df['data'], dayfirst=False) #dayfirst=True
    df.set_index('data', inplace=True)
    uu1 = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    uu2 = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    #df = df.loc[uu1:uu2]
    print(df.head())
    return df

consulta_bc2(12463)  #consulta_bc(12461, '2021-01-04', '2021-01-31')

In [ ]:
def consulta_bc3(codigo_bc, start_date = '02/01/2017', end_date = '12/02/2021' ):   #'2017-01-02'  '2021-02-12'
    import requests
    #url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bc)
    
    #url = ('http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json&dataInicial={}&dataFinal={}')
    url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.'+str(codigo_bc)+'/dados?formato=json&dataInicial='+\
    str(start_date)+'&dataFinal='+str(end_date)

    request_url = url.format(codigo_bc, start_date, end_date)
    r = requests.get(request_url)
    if r.status_code != 200:
        print('Request para API deu erro')  
    try:
        if r.json() != {}:
            print(r)
            print('')
            print(r.json())
            df = pd.DataFrame.from_dict(r.json())
            
            print(df.info())
            df['data'] = pd.to_datetime(df['data'], dayfirst = False, format = '%Y-%m-%d')#format = '%d/%m%Y') #dayfirst = True
            df.reset_index(drop=True)
            df.set_index('data', inplace = True)
            ##index = pd.date_range(start = '2015-01-01', end = '2020-09-01', freq = "M") 
            df.index = pd.to_datetime(df.index)
            df = df.loc[start_date : end_date]
            
        else:
            df = pd.DataFrame()
            print('')
            print("Resposta Inválida")
            time.sleep(1)
    except ValueError:
            print("Resposta Inválida")
            time.sleep(1)
    
    return url, r.json, df 
    

### A consulta a seguir funciona às vezes - API do Bacen não é 100% confiável para mim

In [ ]:
[nome_url, conteudo_json , df_bcb] = consulta_bc3(12461, start_date = '15/01/2021', end_date = '13/02/2021') #'2021-01-04', '2021-01-31')

In [ ]:
#df_bcb.set_index('data', inplace = True)
#df_bcb.info()
#df_bcb.index = pd.to_datetime(df_bcb.index)
#df_bcb.head()

In [ ]:
lista_codigos

In [ ]:
url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.'+str(codigo_bc)+'/dados?formato=json&dataInicial='+\
str(start_date)+'&dataFinal='+str(end_date)
print(url)

## Usando o Quandl

- 12461 = IRF-M (ANBIMA Indice Renda Fixa (pré))
- 12462 = IMA-S (Selic)
- 12463 = IMA-C (IGP-M - será descontinuado em 04/2021)
- 12466 = IMA-B (NTN-B, IPCA)
- 12467 = IMA-B 5 (NTN-B < 5 anos)
- 12468 = IMA-B 5+ 
- 12469 = IMA-Geral
- 17626 = IRF - M1 (pré < 1 ano)
- 17627 = IRF - M1+ (pré > 1 ano)
- 17628 = IMA Geral ex C
Fontes: Abecip e BCB-Depec

In [ ]:
series_bcb = ['IRF-M', 'IMA-S', 'IMA-C', 'IMA-B', 'IMA-B5', 'IMA-B5plus', 'IMA-Geral','IRF-M1', 'IRF-M1plus', 'IMA-Geral_exC']

In [ ]:
time_start = time.time()
time_start;

inicio = '2017-01-01'
fim = '2021-02-11'

bcb_out = [quandl.get('BCB/'+str(serie), start_date = inicio, end_date = fim) for serie in lista_codigos] 


df_bcb = pd.concat(bcb_out, axis = 1)
df_bcb.columns = series_bcb

time_end = time.time();
elapsed_time = time_end - time_start;
print('')
print('*********************************************************************')
print('Demorou:',round(elapsed_time,2),' segundos')
print('*********************************************************************')

In [ ]:
df_bcb.head(3)

In [ ]:
df_bcb.tail(3)

In [ ]:
df_eco.info()

In [ ]:
df_bcb.info()

In [ ]:
df_all =pd.merge(df_eco,df_bcb, how='outer', left_index=True, right_index=True)

In [ ]:
df_all.info()

In [ ]:
df_all.tail(30)

### Cria indicadores de mês e ano para todas as linhas (mesmo onde havia NaN após o merge)

In [ ]:
df_all['MONTH'] = df_all.index.month.astype('int')
df_all['YEAR'] = df_all.index.year.astype('int')

In [ ]:
df_all.info()

In [ ]:
# Define data de hoje (para incluir nos arquivos de saida)
data_hoje = datetime.datetime.now(tz = local_tz)
# Converte para string (incluindo hora e minuto - útil se for rodar várias vezes no mesmo dia para testes)
data_hoje = data_hoje.strftime("%Y_%m_%d_%H%M")

In [ ]:
time_start = perf_counter()
df_all.to_csv(path_result+'fat_risco_'+str(data_hoje)+'.csv',sep = ';' ,encoding='utf-8')

time_end = perf_counter()
elapsed_time = time_end - time_start
print(' ****salvar df em .csv demorou ', round(elapsed_time,2) , ' segundos *****')


In [ ]:
time_start = perf_counter()
df_all.to_pickle(path_result+'fat_risco_'+str(data_hoje)+'.pkl')

time_end = perf_counter()
elapsed_time = time_end - time_start
print(' ****salvar df em .pkl demorou ', round(elapsed_time,2) , ' segundos *****')